# London Bikshare August 2023 Analysis

## Instructions

### Setup

In order to run this Notebook you will need to upload the contents of `computed_data/` to the runtime of Google Colab.

The variables in the next block are used to locate these files.

If the files are in the root of the Colab project then these should work and have been tested. If, for any reason, the location has changed, please update `CALCULATED_DATASET_FOLDER`.

### Sections

The notebook is split into sections replicating spearate analysis files from the project.

In some cases the visualisations are resource-intensive so have been commented out to prevent overloading your instance.

## Main Dashboard

The main component of the final analysis is a Dash application dashboard which allows users to select stations, weather events etc.

The dashboard depends on the files from computed_data and is served with ngrok, a reverse proxy which should make the app visible in google Colab.

This app has been placed at the end of the file to mitiate side effect errors on the other parts of the analysis should anything go wrong with ngrok.

Once finished it is recommended to uncomment the section with `ngrok.kill()` and run it to ensure the app shuts down completely.

In [ ]:
%pip install dash plotly pandas pyngrok dash_bootstrap_components kagglehub

In [ ]:
import threading

from datetime import datetime
from typing import List

import dash_bootstrap_components as dbc
import kagglehub
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from dash import callback, Dash, dcc, html, Input, Output
from pyngrok import ngrok

## File Locations

Defines the locations of the computed files which the Notebook depends on.

These are the computed results of successive calls to the TFL and OSM APIs and so should not be repeated unless ncessary.

To re-generate the data the files prefixed with `setup_{number}` are used.

In [ ]:

# Top-level folder containing the computed datasets.
CALCULATED_DATASET_FOLDER = '.'

# File to hold the result of the TFL API query.
TFL_STATIONS = f'{CALCULATED_DATASET_FOLDER}/tfl-supplied-stations.json'

# File to hold the result of the TFL API query.
TFL_FILTERED = f'{CALCULATED_DATASET_FOLDER}/tfl-supplied-stations-selected.json'

# File to hold the result of the OSM API query.
OSM_STATIONS = f'{CALCULATED_DATASET_FOLDER}/osm-supplied-stations.json'

# File to hold the result of re-queried data from the OSM query.
OSM_FILTERED = f'{CALCULATED_DATASET_FOLDER}/osm-supplied-stations-selected.json'

# File with the combined station data from all sources.
COMBINED_STATIONS = f'{CALCULATED_DATASET_FOLDER}/combined-stations.json'

# File with the combined station data in list / array format.
COMBINED_STATIONS_LIST = f'{CALCULATED_DATASET_FOLDER}/combined-stations-list.json'

TFL_APP_ID = 'edb827fb3ce148daabed9b52433afdc5'
NGROK_TOKEN = '37su9a95SFgEvM3q8sMkNf6SwPb_dAv5YCeUd1ESrobYsM9n'

## Main Datasets Setup & Unique Origin-Destination (OD) Calculation

Downloads and loads in the datasets, sets config variables, and cleans date columns ready for analysis

In [ ]:

ngrok.set_auth_token(NGROK_TOKEN)

# Download latest version of the main dataset.
bike_data_path = kagglehub.dataset_download('kalacheva/london-bike-share-usage-dataset')
weather_data_path = kagglehub.dataset_download('zongaobian/london-weather-data-from-1979-to-2023')

print('Path to bike dataset file:', bike_data_path)
print('Path to weather dataset file:', weather_data_path)

# Load in bike share dataset and weather data
df_bike_data = pd.read_csv(
    f'{bike_data_path}/LondonBikeJourneyAug2023.csv'
)
df_weather_data = pd.read_csv(
    f'{weather_data_path}/london_weather_data_1979_to_2023.csv'
)

# Format date columns to datetime
df_bike_data['Start date'] = pd.to_datetime(
    df_bike_data['Start date'], format='%m/%d/%Y %H:%M'
)
df_bike_data['End date'] = pd.to_datetime(
    df_bike_data['End date'], format='%m/%d/%Y %H:%M'
)

df_bike_data['date_hour'] = df_bike_data['Start date'].dt.floor('h')  # type: ignore
df_bike_data['date_day'] = df_bike_data['Start date'].dt.floor('d')  # type: ignore

df_weather_data['date_formatted'] = pd.to_datetime(df_weather_data['DATE'], format='%Y%m%d')

# Verify the data read was successful.
print(df_bike_data)
print(len(df_bike_data['Start station'].unique()))

# Create a new df_bike_data which contains a list of all unique trips (origin-destination pairs) and the quantity counts for each.
df_unique_od = (
    df_bike_data.groupby(['Start station', 'End station'])
    .size()
    .reset_index()
    .rename(columns={0: 'count'})
)

## Interrogating Unique OD data

In [ ]:
# Take a precursory glance at all trips which only appear once, twice, etc up to 10, assuming a quick drop-off.
# The results are noted in commas following for quick reference, do not assume these are unchanged.
print(df_unique_od)
print('1 trip occurrences: ', df_unique_od['count'].value_counts().get(1))  # 76943
print('2 trip occurrences: ', df_unique_od['count'].value_counts().get(2))  # 36612
print('3 trip occurrences: ', df_unique_od['count'].value_counts().get(3))  # 20516
print('4 trip occurrences: ', df_unique_od['count'].value_counts().get(4))  # 13231
print('5 trip occurrences: ', df_unique_od['count'].value_counts().get(5))  # 9195
print('6 trip occurrences: ', df_unique_od['count'].value_counts().get(6))  # 6403
print('7 trip occurrences: ', df_unique_od['count'].value_counts().get(7))  # 4907
print('8 trip occurrences: ', df_unique_od['count'].value_counts().get(8))  # 3871
print('9 trip occurrences: ', df_unique_od['count'].value_counts().get(9))  # 2988
print('10 trip occurrences: ', df_unique_od['count'].value_counts().get(10))  # 2439
# Total <= 10 trips: 177105
# Total > 10 trips: 191630 - 177105 = 14525

# Take the largest number of repeat trips for reference (York Way, KX).
print('Max repeat values: ', df_unique_od.max())

### Number of trips by start station

In [ ]:
# From vis_highest_repeat_unique_od.py
x_axis_quant = 20
df_sorted = df_unique_od.sort_values('count', ascending=False).reset_index().head(x_axis_quant)
print(df_sorted)

fig = px.bar(df_sorted, x='Start station', y='count')

fig.show()

## Circular Trips and False Starts

For a given start station, defaulted to "York Way, Kings Cross", calculate the number of rides which end at the same station, and make an esitmation of how many of these are "False Starts"; trips which were abandoned early indicating a bike fault.

In [ ]:
target_station = 'York Way, Kings Cross'

# Find a subset of data matching YW, Kings Cross as a start station. This will for the basis of later queries examining the validity of this subset.
print('All from KX: ')
print(df_bike_data.loc[df_bike_data['Start station'] == target_station])

# We assume that trips under 60 seconds are potential "false starts". Select how many fall into this category.
print('All from KX under 1 minute: ')
print(
    df_bike_data.loc[df_bike_data['Start station'] == target_station]['Total duration (ms)']
    .apply(lambda x: x < 60000)
    .sum()
)

# Given very few are potential "false starts", we must assume these are circular trips.
# How many unique OD trips are circular?
print('Unique Start station and End station are the same: ')
print(
    df_unique_od.loc[
        df_unique_od['Start station'] == df_unique_od['End station']
    ].reset_index()
)

# How many trips are circular in the entire dataset?
print('Full DF Start station and End station are the same: ')
print(df_bike_data.loc[df_bike_data['Start station'] == df_bike_data['End station']].reset_index())

# Of this, how many are potential "false starts"?
print('Full DF Start station and End station are the same and less than 60 seconds: ')
print(
    df_bike_data.loc[df_bike_data['Start station'] == df_bike_data['End station']]['Total duration (ms)']
    .apply(lambda x: x < 60000)
    .sum()
)

## Number of Repeat Trips by Count

Aggregates the number of repeat trips vs their count in the dataset.

E.g. 1 would indicate trips which appear only once, 2 is trips with two entries and so on.

Creates a DF to sample some of these and plots the rest on a scatterplot, with a cut off.

In [ ]:
# Create a simple scatter plot of the unique OR trip df.
x = []
y = []

# Given the maximum value from `df_unique_od.max()` set the max x-axis of the chart to 3000. Ignore 0 as there cannot be 0 trips.
for i in range(1, 3000):
    # Loop over the trip quantity, query how many counts there are in the unique OD set.
    x.append(i)
    y_val = int(df_unique_od['count'].value_counts().get(i, 0))
    y.append(y_val)

In [ ]:
# Given the steep x-axis drop off, we'll cut the graph at 145 to focus on the lower end.
cut_off = 145

x_left = x[:cut_off]
x_right = x[cut_off:]
y_left = y[:cut_off]
y_right = y[cut_off:]

# Create a table of the outliers for interest.
outliers = []

for idx, x_val in enumerate(x_right):
    if y_right[idx] != 0:
        outliers.append((x_val, y_right[idx]))

print(f'outliers ({len(outliers)}): ', outliers)

df_outliers = pd.DataFrame(
    outliers, columns=['Number of repetitions', 'Quantity of trips']
)

print('Outliers df:')
print(df_outliers)

df_outliers.to_csv('unique-od-outliers-145-cutoff.csv', index=False)

### Run Visual

In [ ]:
# Show the graph
fig = px.scatter(
    x=x_left,
    y=y_left,
    title='Frequency of Repeat Trips',
    labels={'x': 'Number of repeat trips', 'y': 'Quantity of trips'},
)
fig.show()

## Total Pickups Per Start Station

Visualises the number of total pickups per start station across the entire dataset.

In [ ]:
# From vis_total_pickups_sorted.py
# Create a new df which contains a list of all unique trips (origin-destination pairs) and the quantity counts for each.
df_sorted = (
    df_bike_data.groupby(['Start station'])
    .size()
    .reset_index()
    .rename(columns={0: 'count'})
    .sort_values('count', ascending=False)
    .reset_index()
)
print(df_sorted)

fig = px.bar(df_sorted, x='Start station', y='count')

fig.show()

## Trip Count or Avg Duration vs Precipitation

Depending on comments, plots either the trip count or duration versus Precipitation.


In [ ]:
# From vis_weather_vs_count_or_duration

df_merged = pd.merge(
    left=df_bike_data,
    right=df_weather_data,
    left_on=['date_day'],
    right_on=['date_formatted'],
)

hourly_agg = (
    df_merged.groupby('date_hour')
    # df_bike_data.groupby(['date_hour', 'Start station', 'End station'])
    .agg(
        avg_duration_ms=('Total duration (ms)', 'mean'),
        trip_count=('Number', 'count'),
        rain=('RR', 'mean'),
    )
    .reset_index()
)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=hourly_agg['date_hour'],
        y=hourly_agg['trip_count'],
        mode='lines',
        name='Trips Started',
        yaxis='y1',
    )
)

fig.add_trace(
    go.Scatter(
        x=hourly_agg['date_hour'],
        y=hourly_agg['rain'],
        mode='lines',
        name='Precipitation',
        yaxis='y2',
    )
)

fig.update_layout(
    title='Average Trip Duration and Trip Volume per Hour (August 2023)',
    xaxis=dict(title='hour'),
    yaxis=dict(title='Average Trip Duration (ms)', side='left'),
    yaxis2=dict(title='Number of Trips Started', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
)

fig.show()



## Visualise average trip duration by Time

NOTE: This visualisation is intensive and may slow your Colab / Notebook instance.

Uncomment to re-enable and run with this in mind.

In [ ]:

# df_bike_data.sort_values(by='Start date', inplace=True, ascending=False)
# plot2 = px.scatter(df_bike_data, x='Start date', y='Total duration (ms)')
# plot2.show()

## Main Dash Dashboard Application & Dynamic Visualisation

Runs a dhas app served using ngrok.

Once finished it is recommended to uncomment the next block with `ngrok.kill()` to ensure correct shutdown.

In [ ]:

weather_keys = {
    'TX': 'Daily maximum temperature in 0.1°C.',
    'TN': 'Daily minimum temperature in 0.1°C.',
    'TG': 'Daily mean temperature in 0.1°C.',
    'SS': 'Daily sunshine duration in 0.1 hours.',
    'SD': 'Daily snow depth in 1 cm.',
    'RR': 'Daily precipitation amount in 0.1 mm.',
    'QQ': 'Daily global radiation in W/m².',
    'PP': 'Daily sea level pressure in 0.1 hPa.',
    'HU': 'Daily relative humidity in %.',
    'CC': 'Daily cloud cover in oktas.',
}

df_stations = pd.read_json(COMBINED_STATIONS)

df_bike_data_parsed = pd.merge(
    df_bike_data,
    df_stations,
    left_on=['Start station'],
    right_on=['bikeDataStationName'],
)

# Ensure Start date is parsed as datetime
df_bike_data_parsed['Start date'] = pd.to_datetime(df_bike_data_parsed['Start date'])

print('>>> df_bike_data_parsed')
print(df_bike_data_parsed)
print(df_bike_data_parsed.columns)

# Extract date (day-level)
df_bike_data_parsed['date_hour'] = df_bike_data_parsed['Start date'].dt.floor('h')  # type: ignore
df_bike_data_parsed['date_day'] = df_bike_data_parsed['Start date'].dt.floor('d')  # type: ignore

df_merged = pd.merge(
    left=df_bike_data_parsed,
    right=df_weather_data,
    left_on=['date_day'],
    right_on=['date_formatted'],
)

app = Dash(
    external_stylesheets=[dbc.themes.BOOTSTRAP],
)

test = df_merged['date_day'].unique()
print('>> test', test)

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    'position': 'fixed',
    'top': 0,
    'left': 0,
    'bottom': 0,
    'width': '20%',
    'padding': '2rem 1rem',
    'background-color': '#f8f9fa',
    'font-size': '13px',
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    'margin-left': '18rem',
    'margin-right': '2rem',
    'padding': '2rem 1rem',
}

sidebar = html.Div(
    [
        html.H2('Mode'),
        dcc.RadioItems(
            options=[
                {'value': 'all', 'label': 'All data'},
                {'value': 'station', 'label': 'By start station'},
            ],
            value='all',
            id='radio-mode',
        ),
        dcc.Dropdown(
            df_bike_data['Start station'].unique(),  # type: ignore
            id='dropdown-start_station',
        ),
        dcc.Checklist(
            [{'value': 'use_ratio', 'label': 'Show weather metric as ratio'}],
            value=['use_ratio'],
            id='checkbox-use_ratio',
        ),
        html.Hr(),
        html.H2('Trip Data'),
        dcc.RadioItems(
            options=[
                {'label': 'Trip count', 'value': 'count'},
                {'label': 'Avg. Duration', 'value': 'duration'},
            ],
            value='count',
            id='radio-compare',
        ),
        html.Hr(),
        html.H2('Weather Data'),
        dcc.RadioItems(
            options=[
                {'value': value, 'label': label}
                for value, label in weather_keys.items()
            ],
            value='RR',
            id='radio-weather',
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(
    [
        dcc.RangeSlider(1, 31, 1, value=[1, 31], id='range-date'),
        dcc.Graph(id='graph-line'),
        dcc.Graph(id='graph-scatter'),
    ],
    style=CONTENT_STYLE,
)

app.layout = [
    html.H1(
        children='Weather events vs Trips whole month', style={'textAlign': 'center'}
    ),
    sidebar,
    content,
]

print('>>> df_merged')
print(df_merged)
print(df_merged.columns)


@callback(
    Output('graph-line', 'figure'),
    Output('graph-scatter', 'figure'),
    Input('radio-compare', 'value'),
    Input('radio-weather', 'value'),
    Input('radio-mode', 'value'),
    Input('dropdown-start_station', 'value'),
    Input('checkbox-use_ratio', 'value'),
    Input('range-date', 'value'),
)
def graph(
    compare_mode: str,
    weather: str,
    mode: str,
    start_station: str,
    use_ratio: List[str],
    range_date: List[int],
):
    start_date = datetime(2023, 8, range_date[0])
    end_date = datetime(2023, 8, range_date[1], 23, 59, 59)

    print('start date: ', start_date)
    print('end date: ', end_date)

    date_mask = (df_merged['date_day'] >= start_date) & (
        df_merged['date_day'] <= end_date
    )

    if mode == 'station':
        mask = (df_merged['Start station'] == start_station) & date_mask
        dff = df_merged[mask]
    else:
        dff = df_merged[date_mask]

    y1_label = (
        'Average Trip Duration (ms)' if compare_mode == 'duration' else 'Trips Started'
    )
    trip_key = 'avg_duration_ms' if compare_mode == 'duration' else 'trip_count'

    print('>>> dff')
    print(dff)
    print(dff.columns)

    hourly_agg = (
        dff.groupby('date_hour')
        .agg(
            avg_duration_ms=('Total duration (ms)', 'mean'),
            trip_count=('Number', 'count'),
            weather_agg=(weather, 'mean'),
        )
        .reset_index()
    )

    hourly_agg['ratio'] = hourly_agg[trip_key] / hourly_agg['weather_agg']

    fig_line = go.Figure()

    fig_line.add_trace(
        go.Scatter(
            x=hourly_agg['date_hour'],
            y=hourly_agg[trip_key],
            mode='lines',
            name=y1_label,
            yaxis='y1',
        )
    )

    fig_line.add_trace(
        go.Scatter(
            x=hourly_agg['date_hour'],
            y=hourly_agg['ratio' if 'use_ratio' in use_ratio else 'weather_agg'],
            mode='lines',
            name=weather_keys[weather],
            yaxis='y2',
        )
    )

    fig_line.update_layout(
        title='Average Trip Duration and Trip Volume per Hour (August 2023)',
        xaxis=dict(title='hour'),
        yaxis=dict(title=y1_label, side='left'),
        yaxis2=dict(title=weather_keys[weather], overlaying='y', side='right'),
        legend=dict(x=0.01, y=0.99),
    )

    fig_scatter = px.scatter_map(
        df_stations,
        lat='lat',
        lon='lon',
        hover_data=['name', 'lat', 'lon'],
        zoom=11,
    )

    if mode == 'station':
        df_scatter_filter = dff[dff['Start station'] == start_station]
        df_scatter_grouped = (
            df_scatter_filter.groupby(['Start station', 'End station'])
            .size()
            .reset_index()
            .rename(columns={0: 'count'})
        )
        df_scatter = pd.merge(
            df_scatter_grouped,
            df_stations,
            left_on=['End station'],
            right_on=['bikeDataStationName'],
        )

        start_station_data = df_stations[
            df_stations['bikeDataStationName'] == start_station
        ].reset_index()
        if len(start_station_data):
            start_station_data = start_station_data.iloc[0]

            for i in range(len(df_scatter)):
                fig_scatter.add_trace(
                    go.Scattermap(
                        lon=[start_station_data['lon'], df_scatter['lon'][i]],
                        lat=[start_station_data['lat'], df_scatter['lat'][i]],
                        mode='lines',
                        opacity=1,
                    )
                )

        pass

    return fig_line, fig_scatter

# Start the app in a thread
def run_app():
    app.run()

thread = threading.Thread(target=run_app)
thread.start()

# Open ngrok tunnel
public_url = ngrok.connect(8050)
print('App is live at:', public_url)


Shut-down the ngrok reverse proxy

In [ ]:
# ngrok.kill()